# 아래 코드는 데이터셋 분리 및 학습결과가 틀린 코드입니다. 빠른 시일 내에 수정한 코드를 다시 올리도록 하겠습니다. 
### 6. 프로젝트 : 작사가 인공지능 만들기
##### !. Google colab pro를 사용하여 작성되었습니다.


---

In [ ]:
#colab 끊김 방지.
#python에서 실행하는 것이 아닌 F12 를 눌러서 console 창에 아래 코드를 복붙해놓으면 계속 특정 시간이 지날 때마다 colab-toolbar-button 을 클릭해주어 런타임 종료를 막아줌.
function ClickConnect(){  
    console.log("코랩 연결 끊김 방지");   
    document.querySelector("colab-toolbar-button#connect").click()   
}  
setInterval(ClickConnect, 60 * 1000)  

In [3]:
#코랩과 구글 드라이브를 연결하기 위함.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---

### (본문)

#### step 1.데이터  및 주요 라이브러리 import

In [4]:
# 곡다운로드
# Kanye_West등 49개 (출처:https://www.kaggle.com/datasets/paultimothymooney/poetry)

In [5]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위함.) 
import numpy as np         # 변환된 문장 데이터(행렬) 처리용.
import tensorflow as tf    # 텐서플로우
import glob
import os #os 사용할 때 경로 상에 특수문자가 있으면 오류발생 .
from sklearn.model_selection import train_test_split
!pip list | grep glob
print(tf.__version__)

glob2                         0.7
2.8.0


#### step 2.데이터 읽어오기

In [6]:
#txt_file_path = os.getenv('HOME') +'/content/drive/MyDrive/Colab Notebooks/E_06_NL/lyricist'
#print(txt_file_path)


In [7]:


txt_file_path = '/content/drive/MyDrive/Colab Notebooks/E_06_NL/lyricist/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:5])

데이터 크기: 187088
Examples:
 ["Let's stay together I, I'm I'm so in love with you", 'Whatever you want to do', 'Is all right with me', 'Cause you make me feel so brand new', "And I want to spend my life with you Let me say that since, baby, since we've been together"]


In [8]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜀.
    #if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜀.

    if idx > 9: break   # 일단 문장 10개만 확인.
        
    print(sentence)

Let's stay together I, I'm I'm so in love with you
Whatever you want to do
Is all right with me
Cause you make me feel so brand new
And I want to spend my life with you Let me say that since, baby, since we've been together
Loving you forever
Is what I need
Let me, be the one you come running to
I'll never be untrue Oh baby
Let's, let's stay together (gether)


#### step 3.데이터 정제

In [9]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀜.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스(공백) 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여줌.
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인.

<start> this is sample sentence . <end>


In [10]:
corpus = [] #여기에 정제된 문장을 모음

for sentence in raw_corpus: #우리가 원하지 않는 문장은 건너뜀
    if len(sentence) == 0: continue
    #if sentence[-1] == ":": continue

    # 정제를 하고 담아줌
    #preprocessed_sentence = preprocess_sentence(sentence)
    #corpus.append(preprocessed_sentence)
    corpus.append(preprocess_sentence(sentence))

# 정제된 결과를 10개만 확인     
corpus[:10]

['<start> let s stay together i , i m i m so in love with you <end>',
 '<start> whatever you want to do <end>',
 '<start> is all right with me <end>',
 '<start> cause you make me feel so brand new <end>',
 '<start> and i want to spend my life with you let me say that since , baby , since we ve been together <end>',
 '<start> loving you forever <end>',
 '<start> is what i need <end>',
 '<start> let me , be the one you come running to <end>',
 '<start> i ll never be untrue oh baby <end>',
 '<start> let s , let s stay together gether <end>']

In [11]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용.
# 더 잘 알기 위해 아래 문서들을 참고하면 좋음.
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 12000단어를 기억할 수 있는 tokenizer를 제작.
        filters=' ',    # 별도로 전처리 로직을 추가. 이번에는 사용하지 않음.(이미 문장을 정제했기 때문)
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지 12000단언에 포함되지 못한 단어는 '<unk>'로 바꿈.
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer 내부의 단어장을 완성

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축.
    tensor = tokenizer.texts_to_sequences(corpus)   # 준비한 tokenizer의 구축한 사전으로부터 corpus를 해석해 Tensor로 변환.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공.
    # maxlen의 디폴트값은 None. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰짐.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[ 16 222 282 ...  31   7   3]
 [  2 598   7 ...   0   0   0]
 [  2  26  24 ...   0   0   0]
 ...
 [  2 100   6 ...   0   0   0]
 [  2 122   9 ...   0   0   0]
 [  2  74   6 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f2f939ab690>


In [12]:
corpus_15 = []
for i in corpus:
    if len(i.split()) <= 15:
        corpus_15.append(i)
corpus_15

['<start> whatever you want to do <end>',
 '<start> is all right with me <end>',
 '<start> cause you make me feel so brand new <end>',
 '<start> loving you forever <end>',
 '<start> is what i need <end>',
 '<start> let me , be the one you come running to <end>',
 '<start> i ll never be untrue oh baby <end>',
 '<start> let s , let s stay together gether <end>',
 '<start> lovin you whether , whether <end>',
 '<start> times are good or bad , happy or sad <end>',
 '<start> oh , oh , oh , oh , yeah <end>',
 '<start> then turn around and make up <end>',
 '<start> i just can t see <end>',
 '<start> you d never do that to me would you , baby <end>',
 '<start> staying around you is all i see <end>',
 '<start> loving you whether , whether <end>',
 '<start> times are good or bad , happy or sad <end>',
 '<start> come on <end>',
 '<start> let s stay , let s stay together let s stay together <end>',
 '<start> was everything a man could want to do <end>',
 '<start> how can you stop the rain from fall

In [13]:
tensor, tokenizer = tokenize(corpus_15)

[[  2 570   7 ...   0   0   0]
 [  2  26  25 ...   0   0   0]
 [  2  66   7 ...   0   0   0]
 ...
 [  2 100   6 ...   0   0   0]
 [  2 116   9 ...   0   0   0]
 [  2  73   6 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f2f939ab650>


In [14]:
print(tensor.shape)
print(tensor[:3, :10]) #생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력.

(156227, 15)
[[  2 570   7  64  10  48   3   0   0   0]
 [  2  26  25  87  30  12   3   0   0   0]
 [  2  66   7  72  12 106  31 810 153   3]]


In [15]:
#텐서 데이터는 모두 정수로 이루어져 있으.ㅁ
# 이 숫자는 tokenizer에 구축된 단어 사전의 인덱스.
# 단어 사전이 어떻게 구축되었는지 아래와 같이 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


#### step 4. 평가 데이터셋 분리

In [16]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음.
tgt_input = tensor[:, 1:]   # tensor에서 <start>를 잘라내서 타겟 문장을 생성.

print(src_input[0])
print(tgt_input[0])

[  2 570   7  64  10  48   3   0   0   0   0   0   0   0]
[570   7  64  10  48   3   0   0   0   0   0   0   0   0]


In [17]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=8)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)


In [18]:
enc_train, enc_val, dec_train, dec_val


(array([[   2,    4,  164, ...,    0,    0,    0],
        [   2, 1158,  971, ...,    0,    0,    0],
        [   2, 1449,    9, ...,    0,    0,    0],
        ...,
        [   2,  127,    6, ...,   11, 2254,    3],
        [   2,   79,  302, ...,    0,    0,    0],
        [   2,  674,    6, ...,    0,    0,    0]], dtype=int32),
 array([[   2,  563,    9, ...,    0,    0,    0],
        [   2,    4,   61, ...,    0,    0,    0],
        [   2, 1680,  109, ...,    0,    0,    0],
        ...,
        [   2, 1966,    1, ...,    3,    0,    0],
        [   2,    4,   24, ...,    0,    0,    0],
        [   2,    8,    4, ...,  166,   12,    3]], dtype=int32),
 array([[   4,  164,  275, ...,    0,    0,    0],
        [1158,  971, 9926, ...,    0,    0,    0],
        [1449,    9, 3249, ...,    0,    0,    0],
        ...,
        [ 127,    6,  981, ..., 2254,    3,    0],
        [  79,  302, 2944, ...,    0,    0,    0],
        [ 674,    6, 1174, ...,    0,    0,    0]], dtype=int32)

In [19]:
# tf
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
# 준비한 데이터 소스로부터 데이터셋을 만듬.
# 데이터셋에 대해서는 아래 문서를 참고.
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서.
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
#dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
#dataset = dataset.shuffle(BUFFER_SIZE)
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

#### step 5. 인공지능 만들기

In [29]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 #워드 베겉의 차원수, 즉 단어가 추상적으로 표현되는 크기.
#그 크기가 2라면 예를 들어
# 차갑다: [0.0, 1.0]
# 뜨겁다: [1.0, 0.0]
# 미지근하다: [0.5, 0.5] 정도의 구분이 가능.
hidden_size = 1024 # LSTM 레이어의 hidden state의 차원수. hidden_size는 모델에 얼마나 많은 일꾼을 둘 것인가?로 이해해도 무방.
# 모두 같은 데이터를 보고 각자의 생각을 가지는데, 역시 충분한 데이터 주어지면 올바른 결정을 내림. 그렇지 않으면 배가 산으로 감. 
# 이번 프로젝트에서는 1024가 적당.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [21]:
#데이터셋에서 데이터 한 배치만 불러오는 방법.
# 동작원리는 어려우니 차후 공부 필요.
for src_sample, tgt_sample in dataset.take(1): break
# 한 배치만 불러온 데이터를 모델에 넣기.
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.1825029e-05, -1.1227794e-04, -6.1600564e-05, ...,
         -9.9282197e-06, -1.9726263e-04, -2.6309502e-04],
        [ 1.9305773e-04, -2.6407777e-04, -4.9573288e-04, ...,
         -4.6306748e-05, -6.6772074e-05, -4.3870689e-04],
        [ 2.3568604e-04, -3.6936003e-04, -4.3251569e-04, ...,
          4.8879916e-05,  3.0254593e-04, -5.3413276e-04],
        ...,
        [-1.4541956e-03,  1.1111943e-03,  4.5771786e-04, ...,
         -6.7979452e-04, -2.9742776e-03, -2.2640200e-03],
        [-1.6493879e-03,  1.0640661e-03,  5.5047683e-04, ...,
         -7.7940739e-04, -3.6162729e-03, -2.3913682e-03],
        [-1.8216551e-03,  1.0171460e-03,  6.4120244e-04, ...,
         -8.7969931e-04, -4.1889418e-03, -2.4897621e-03]],

       [[ 1.1825029e-05, -1.1227794e-04, -6.1600564e-05, ...,
         -9.9282197e-06, -1.9726263e-04, -2.6309502e-04],
        [-9.3757131e-05,  3.1967204e-05, -1.4260611e-04, ...,
         -1.5939320e-04, 

In [22]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [23]:
#loss 2.2 이하가 나와야함. -> 1.14 달성
# optimizer와 loss와 관련된 자세한 내용은 아래 문서 참고
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 굉장히 오래 걸리는 작업
# 그래도 혹시라도 학습에 지나치게 많은 시간이 소요된다면 tf.test.is_gpu_available() 소스를 실행해 텐서플로우가 GPU를 잘 사용하고 있는지 확인
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
610/610 [==============================] - 57s 89ms/step - loss: 3.4128
Epoch 2/30
610/610 [==============================] - 54s 89ms/step - loss: 2.9737
Epoch 3/30
610/610 [==============================] - 54s 89ms/step - loss: 2.8006
Epoch 4/30
610/610 [==============================] - 54s 89ms/step - loss: 2.6688
Epoch 5/30
610/610 [==============================] - 54s 89ms/step - loss: 2.5549
Epoch 6/30
610/610 [==============================] - 54s 89ms/step - loss: 2.4529
Epoch 7/30
610/610 [==============================] - 54s 89ms/step - loss: 2.3587
Epoch 8/30
610/610 [==============================] - 54s 89ms/step - loss: 2.2704
Epoch 9/30
610/610 [==============================] - 54s 89ms/step - loss: 2.1872
Epoch 10/30
610/610 [==============================] - 54s 89ms/step - loss: 2.1088
Epoch 11/30
610/610 [==============================] - 54s 89ms/step - loss: 2.0341
Epoch 12/30
610/610 [==============================] - 54s 89ms/step - loss: 1.9634
E

In [24]:
#모델 평가용 코드.
# generate_text 함수를 만들어서 모델에게 시작문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행.
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환함.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 함.
    while True:
        predict = model(test_tensor)  # 1. 입력받은 문장의 텐서를 입력함. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 2. 예측된 값 중 가장 높은 확률인 word index를 봅아냄. 즉, 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됨.

        # 3. 2에서 예측된 word index를 문장뒤에 붙임. 즉, 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줌.
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측. max_len에 도달했다면 문장 생성을 마침.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 최종적으로 모델이 생성한 자연어 문장값을 돌려줌.

In [28]:
# generate_text() 함수에서 init_sentence를 인자로 받고 있는 중.
# 이렇게 받은 인자를 일단 텐서로 만들고 있음.
# 디폴트로는 <start>단어 하나만 받는 중.
# while의 첫 번째 루프에서 test_tensor에 <start> 하나만 들어갔다고 침. 우리의 모델이 출력으로 12001개의 단어 중 A를 골랐다고 침.
# while의 두 번째 루프에서 test_tensor에는 <start> A가 들어감. 그래서 우리의 모델이 그다음 B를 골랐다고 침.
# while의 세 번째 루프에서 test_tensor에는 <start> A B가 들어감. (이하 후략)
# 실제로 위 문장 생성 함수를 실행.
generate_text(model, tokenizer, init_sentence="<start> love") #시작 단어를 love 사용


'<start> love me , love me yeah come here rude boy , boy <end> '

입력값: love -> love me , love me yeah come here rude boy , boy  
사랑에서 많은 일을 겪었던 것 같은 중2병 인공지능이 나왔다.   
들어간 데이터들이 대부분 서정적인 부분이 많기 때문에 인공지능도 덩달아 감성적으로 나온 것 같다.  

In [26]:
# init_sentence를 바꿔가면 실험. <start>는 필수.
generate_text(model, tokenizer, init_sentence="<start> earth") #시작 단어를 earth 사용


'<start> earth is chillin <end> '

입력값: earth -> earth is chillin  
말은되지만 문장 자체는 완성도가 낮다고 판단된다. 들어간 데이터가 대부분 사랑과 관련된 노래이기 때문인 것 같다.  

In [27]:
# init_sentence를 바꿔가면 실험. <start>는 필수.
generate_text(model, tokenizer, init_sentence="<start> AI") #시작 단어를 AI 사용

'<start> ai <end> '

입력값: AI -> ai  
ai와 관련된 단어는 데이터는 없어서인지 학습하지 못했고 아무런 문장도 만들어내지 못했다.

---

### 평가기준.

1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?  O
텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?  O
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?  O
특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?  O
3. 텍스트 생성모델이 안정적으로 학습되었는가?  O
텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?  O


### 회고록.


문자데이터가 담긴 사랑노래 49개의 데이터셋의 문장데이터를 re 라이브러리를 사용하여 데이터를 다듬은 후 인공지능에게 학습시키기 위해 keras 모델을 subcalssing하여 텍스트 분류 모델을 만든 후 인공지능이 적절한 가사를 만들어냈는지 확인해보는 6번째 Exploration 이었다.  
연극대사 데이터셋 대신 캐글에 비슷하게 분석해 볼 수 있는 노래 가사 데이터셋이 있어서 사용했더니 모든 단어를 사랑과 관련짓는 사랑꾼 인공지능이 만들어져 신기하기도 했다.  
인공지능에서 데이터셋이 왜 중요한지 다시한번 알 수 있는 실습이였다.  


새로운 문장을 만들기 위해서는 재학습을 시켰어야 했는데 학습의 시간이 개인컴퓨터로는 24시간이 걸려도 끝내지 못했고 그렇다고 epoch값을 낮추니 문장이 형편없이 나왔다.   
나중에 tf.test.is_gpu_available()로 확인해보니 개인컴퓨터에서 gpu를 제대로 사용하지 못하고 있었다.  
학습수준은 최대한 좋게 유지하면서 분석의 효율을 높이기 위해 colab을 사용했다.  
전에 겪었던 상황인 colab의 런타임 끊김을 방지하기 위해  아래와 같은 코드를 구글링을 통해 찾아냈다.  
'''  
function ClickConnect(){  
    console.log("코랩 연결 끊김 방지");   
    document.querySelector("colab-toolbar-button#connect").click()   
}  
setInterval(ClickConnect, 60 * 1000)  
'''  
웹페이지에 코드를 추가하는 방법을 찾아내어 적용했고 다행히 학습 중 끊김없이 빠르게 끝낼 수 있었다.  
시간이 단축되니 여러번 학습을 돌렸고 하이퍼 파라미터를 재조정해 나온 다음과 같이 문장을 출력할 수 있었다.  
 <start> love is a losing game it s all right <end>  
 <start> earth is an eenie meenie miney mo lover <end>  

 데이터가 그대로이다 보니 분석결과 대부분이 사랑과 관련된 출력문장들이었지만 추후 데이터셋을 변경 힙합같은 여러 장르에 음악도 도전해보고 싶다.  
학습이 잘 안된다면 '래원'같은 결과가 나오겠지만 잘 된다면 'jk 타이거'같은 결과가 나오지 않을까싶다.

이번 exp를 진행하면서 코드를 완전히 이해한 것은 아니었다.  
그러나 추후 한국어 문장을 만들어내기 위해서는 문장학습(RNN, 토큰같은)에 대한 개념을 좀 더 이해할 필요가 있음.



----

### Reference. (APA)

조태호. (2020). 모두의 딥러닝. 길벗.  
이영호. (2020). 모두의 인공지능 with 파이썬. 길벗.


-----